# Jeremy Romanowski's IBM Applied Data Science Capstone Project
## The Battle of Seattle's Neighborhoods

**Project Summary:**<br>
In this project we're going to map and analyze the various neighborhoods of Seattle, Washington USA. It is the aim of this project to find the most ideal location(s) for a new coffee franchise

***1) Importing Dependencies:***

In [2]:
pip install folium==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas==1.2.0

  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.1
    Uninstalling pandas-1.0.1:
      Successfully uninstalled pandas-1.0.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import folium

In [3]:
import pandas as pd # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
print(pd. __version__)

1.2.0


In [4]:
import numpy as np 
import json 
from geopy.geocoders import Nominatim 
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


## Import the Seattle Neighborhood data

In [5]:
with open('seattle_neighborhood_data.json') as json_data:
    seattle_data = json.load(json_data)
    
print("Data loaded!")

Data loaded!


### Here's the data loaded into the variable **seattle_data**:

In [6]:
seattle_data

{'type': 'FeatureCollection',
 'totalFeatures': 22,
 'features': [{'properties': {'type': 'Feature',
    'postal_code': '98003',
    'neighborhood': 'Federal Way'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98005',
    'neighborhood': 'Bellevue'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98033',
    'neighborhood': 'Kirkland'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98037',
    'neighborhood': 'Lynnwood'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98040',
    'neighborhood': 'Mercer Island'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98052',
    'neighborhood': 'Redmond'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98055',
    'neighborhood': 'Renton'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98101',
    'neighborhood': 'Downtown'}},
  {'properties': {'type': 'Feature',
    'postal_code': '98102',
    'neighborhood': 'Capitol Hill'}},
  {'properties': {'type': 'Feature',

#### The neighborhood data has already been cleaned; all data is found in the *'features'* key, so let's isolate that data now:

In [7]:
neighborhoods_data = seattle_data['features']

### Let's look at the first few entries:

In [8]:
neighborhoods_data[0:5]

[{'properties': {'type': 'Feature',
   'postal_code': '98003',
   'neighborhood': 'Federal Way'}},
 {'properties': {'type': 'Feature',
   'postal_code': '98005',
   'neighborhood': 'Bellevue'}},
 {'properties': {'type': 'Feature',
   'postal_code': '98033',
   'neighborhood': 'Kirkland'}},
 {'properties': {'type': 'Feature',
   'postal_code': '98037',
   'neighborhood': 'Lynnwood'}},
 {'properties': {'type': 'Feature',
   'postal_code': '98040',
   'neighborhood': 'Mercer Island'}}]

## Transforming the data into a Pandas dataframe:

In [9]:
# define the dataframe columns
column_names = ['postal_code', 'neighborhood'] 

# instantiate the dataframe
sea_neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
sea_neighborhoods

,postal_code,neighborhood


### Now we're going to for loop through the data and populate the empty dataframe:

In [11]:
for data in neighborhoods_data:
    postal_code = data['properties']['postal_code'] 
    neighborhood = data['properties']['neighborhood']
    
    sea_neighborhoods = sea_neighborhoods.append({'postal_code': postal_code, 
                                          'neighborhood': neighborhood
                                         }, ignore_index=True)

### Here's the first 10 entries of the resulting dataframe:

In [12]:
sea_neighborhoods

,postal_code,neighborhood
0,98003,Federal Way
1,98005,Bellevue
2,98033,Kirkland
3,98037,Lynnwood
4,98040,Mercer Island
5,98052,Redmond
6,98055,Renton
7,98101,Downtown
8,98102,Capitol Hill
9,98103,"Greenwood, Fremont, Greenlake"


### And finally, let's check the size of the dataframe:

In [13]:
sea_neighborhoods.shape

(22, 2)

### Now we're going to load the Seattle geopy CSV data into a Pandas dataframe:

In [14]:
coords = pd.read_csv('geopy_seattle_coords.csv')

### Let's take a look at the head of the Seattle neighborhood coordinates:

In [15]:
coords.head(10)

,postal_code,latitude,longitude
0,98003,47.3223,-122.3126
1,98005,47.6101,-122.2015
2,98033,47.6769,-122.2060
3,98037,47.8209,-122.3151
4,98040,47.5707,-122.2221
5,98052,47.6740,-122.1215
6,98055,47.4797,-122.2079
7,98101,47.6110,-122.3330
8,98102,47.6253,-122.3222
9,98103,47.6907,-122.3553


In [16]:
coords.shape

(22, 3)

### Excellent, the dataframe index shape is consistent. Now we're going to merge the neighborhoods dataframe with the geopy coordinates csv dataframe:

Before we can get started with merging, we need to verify the data types in each dataframe are compatible with one another:

In [17]:
coords.dtypes

postal_code      int64
latitude       float64
longitude      float64
dtype: object

In [18]:
sea_neighborhoods.dtypes

postal_code     object
neighborhood    object
dtype: object

As we can see, the data types are not consistent between the Postal Code columns -- we need both of them to be treated as a string, rather than an integer value. This ensures we can use onehot encoding effectively, and other useful categorical analyses.

In [23]:
# Let's convert the Postal Codes in the coords dataframe to type string:
coords['postal_code'] = coords.postal_code.astype(str)

In [24]:
# Let's verify the datatype has been changed to type string:
coords.dtypes

postal_code     object
latitude       float64
longitude      float64
dtype: object

### Excellent, we now have compatible 'postal_code' datatypes and can proceed with merging the two dataframes:

In [25]:
neighborhoods_with_coords = sea_neighborhoods.merge(coords, on='postal_code')

In [26]:
neighborhoods_with_coords

,postal_code,neighborhood,latitude,longitude
0,98003,Federal Way,47.3223,-122.3126
1,98005,Bellevue,47.6101,-122.2015
2,98033,Kirkland,47.6769,-122.2060
3,98037,Lynnwood,47.8209,-122.3151
4,98040,Mercer Island,47.5707,-122.2221
5,98052,Redmond,47.6740,-122.1215
6,98055,Renton,47.4797,-122.2079
7,98101,Downtown,47.6110,-122.3330
8,98102,Capitol Hill,47.6253,-122.3222
9,98103,"Greenwood, Fremont, Greenlake",47.6907,-122.3553


### Now we're going to create a map of Seattle with neighborhoods superimposed on top to visualize the data better:

In [27]:
# The coordinates of Seattle are as follows: 47.6062° N, -122.3321° W -- these are the lat/long values we will use to center our map:
latitude = 47.6062
longitude = -122.3321

seattle_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Now we have a map location and Folium map object created, let's add markers to the map!

for lat, long, postal_code, neighborhood in zip(neighborhoods_with_coords['latitude'], neighborhoods_with_coords['longitude'], neighborhoods_with_coords['postal_code'], neighborhoods_with_coords['neighborhood']):
    label = '{}, {}'.format(neighborhood, postal_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#33FFF2',
        fill_opacity=0.75).add_to(seattle_map)
    
seattle_map

# Looks good!

### Now we're going to start utilizing the Foursquare API to explore the neighborhoods and segment them. To begin, we need to define the credentials for the Foursquare API calls:

In [35]:
CLIENT_ID = 'REDACTED' # Foursquare ID
CLIENT_SECRET = 'REDACTED' # Foursquare Secret
VERSION = '20180605' # API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: REDACTED
CLIENT_SECRET:REDACTED


### Now let's get the top 100 venues that are within a 2000 meter radius:

In [32]:
radius = 2000
LIMIT = 100

# Let's create an empty container for the venues:
seattle_venues = []

for lat, long, post, neighborhood in zip(neighborhoods_with_coords['latitude'], neighborhoods_with_coords['longitude'], neighborhoods_with_coords['postal_code'], neighborhoods_with_coords['neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        seattle_venues.append((
            post, 
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

### Now we're going to convert the seattle_venues list to a pandas dataframe:

In [23]:
seattle_venues_df = pd.DataFrame(seattle_venues)

# Let's get some column names populated:
seattle_venues_df.columns = ['Postal Code', 'Neighborhood', 'NeighborhoodLat', 'NeighborhoodLong', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

# Let's check out the head:
seattle_venues_df.head(10)

,Postal Code,Neighborhood,NeighborhoodLat,NeighborhoodLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,98003,Federal Way,47.3223,-122.3126,Barnes & Noble,47.320866,-122.315078,Bookstore
1,98003,Federal Way,47.3223,-122.3126,Tokyo-ya Ramen & Izakaya,47.319753,-122.315029,Soup Place
2,98003,Federal Way,47.3223,-122.3126,H Mart,47.321783,-122.314377,Grocery Store
3,98003,Federal Way,47.3223,-122.3126,Traditional Korean Beef Soup 전통설렁탕,47.321057,-122.312627,Korean Restaurant
4,98003,Federal Way,47.3223,-122.3126,Los Bigotes De Villa,47.328793,-122.312070,Mexican Restaurant
5,98003,Federal Way,47.3223,-122.3126,Grand Peking,47.316447,-122.311561,Chinese Restaurant
6,98003,Federal Way,47.3223,-122.3126,85C Bakery Cafe,47.319851,-122.315143,Bakery
7,98003,Federal Way,47.3223,-122.3126,Total Wine & More,47.312917,-122.315603,Wine Shop
8,98003,Federal Way,47.3223,-122.3126,Chick-fil-A,47.314190,-122.313991,Fast Food Restaurant
9,98003,Federal Way,47.3223,-122.3126,"iLoveKickboxing - Federal Way, WA",47.312313,-122.316110,Gym


### Let's check how many venues were returned for each Postal Code:

In [24]:
seattle_venues_df.groupby(['Postal Code', 'Neighborhood']).count()

,,NeighborhoodLat,NeighborhoodLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal Code,Neighborhood,,,,,,
98003,Federal Way,100,100,100,100,100,100
98005,Bellevue,100,100,100,100,100,100
98033,Kirkland,92,92,92,92,92,92
98037,Lynnwood,100,100,100,100,100,100
98040,Mercer Island,41,41,41,41,41,41
98052,Redmond,100,100,100,100,100,100
98055,Renton,100,100,100,100,100,100
98101,Downtown,100,100,100,100,100,100
98102,Capitol Hill,100,100,100,100,100,100


## Let's examine each area with their respective venue categories:

In [25]:
# First, let's one-hot encode all of these:
seattle_onehot = pd.get_dummies(seattle_venues_df[['VenueCategory']], prefix="", prefix_sep="")

# Now let's add the columns of interest (neighborhood, postal code, borough) into this dataframe:
seattle_onehot['Neighborhood'] = seattle_venues_df['Neighborhood']
seattle_onehot['Postal Code'] = seattle_venues_df['Postal Code']

In [26]:
# Let's see what the seattle_onehot dataframe looks like:
seattle_onehot.head(10)

,ATM,African Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Boat Launch,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighborhood,Postal Code
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

In [27]:
# Let's also check out the shape:
seattle_onehot.shape

(2059, 276)

### It's now time to group rows by neighborhood and take the mean of the frequency of each category's occurence:

In [28]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,ATM,African Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Boat Launch,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bainbridge Island,0.000000,0.00,0.00,0.016667,0.00,0.00,0.00000,0.016667,0.000000,0.00000

In [29]:
# Checking the new size:
seattle_grouped.shape

(22, 275)

### Excellent -- we've successfully populated the 22 Seattle neighborhoods with the frequency of venue type

### Now we can print each neighborhood with its top 5 most common venues:

In [30]:
number_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(number_top_venues))
    print('\n')

----Bainbridge Island----
            venue  freq
0   Boat or Ferry  0.08
1     Pizza Place  0.05
2     Coffee Shop  0.05
3  Ice Cream Shop  0.03
4           Diner  0.03


----Ballard----
                venue  freq
0         Coffee Shop  0.06
1  Mexican Restaurant  0.05
2        Cocktail Bar  0.05
3                 Bar  0.05
4      Ice Cream Shop  0.04


----Bellevue----
              venue  freq
0       Coffee Shop  0.05
1       Yoga Studio  0.03
2  Toy / Game Store  0.03
3        Steakhouse  0.03
4    Clothing Store  0.03


----Belltown----
            venue  freq
0     Coffee Shop  0.08
1           Hotel  0.08
2          Bakery  0.06
3    Cocktail Bar  0.03
4  Breakfast Spot  0.03


----Capitol Hill----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2              Bakery  0.05
3                Café  0.04
4  Italian Restaurant  0.03


----Columbia City----
                   venue  freq
0  Vietnamese Restaurant  0.08
1            Pizza Place  0.0

### Let's move this data into a dataframe, starting with a function to sort the venues in descending order:

In [31]:
def return_most_common_venues(row, number_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:number_top_venues]

### Now let's create the new dataframe and check out the top 10 venues for each neighborhood:

In [32]:
number_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(number_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], number_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bainbridge Island,Boat or Ferry,Pizza Place,Coffee Shop,Trail,Bank,Sandwich Place,Ice Cream Shop,Park,Diner,Scenic Lookout
1,Ballard,Coffee Shop,Cocktail Bar,Mexican Restaurant,Bar,Ice Cream Shop,Bakery,French Restaurant,Thai Restaurant,Park,Brewery
2,Bellevue,Coffee Shop,Yoga Studio,Toy / Game Store,Steakhouse,Shopping Mall,Clothing Store,Spa,Bookstore,Boutique,Bubble Tea Shop
3,Belltown,Hotel,Coffee Shop,Bakery,Movie Theater,Breakfast Spot,Cocktail Bar,Sculpture Garden,Deli / Bodega,Middle Eastern Restaurant,Scenic Lookout
4,Capitol Hill,Coffee Shop,Bakery,Cocktail Bar,Café,Italian Restaurant,Sandwich Place,Park,Yoga Studio,Museum,Bar


# Finally, let's start clustering the neighborhoods! We will be using K-Means clustering, with a cluster size of 5:

In [33]:
# Number of k-means clusters:
kclusters = 5

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 0, 3, 3, 2, 3, 0, 1, 3], dtype=int32)

### Now we're going to create a new dataframe that includes the cluster number in addition to the top 10 venues for each neighborhood:

In [34]:
# Start by adding cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

seattle_merged = neighborhoods_with_coords.copy()

# Merging toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
seattle_merged = seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

seattle_merged.head() # Checking the head

,postal_code,neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,98003,Federal Way,47.3223,-122.3126,0,Mexican Restaurant,Korean Restaurant,Coffee Shop,Grocery Store,Burger Joint,Pet Store,Ice Cream Shop,Department Store,Thai Restaurant,Vietnamese Restaurant
1,98005,Bellevue,47.6101,-122.2015,0,Coffee Shop,Yoga Studio,Toy / Game Store,Steakhouse,Shopping Mall,Clothing Store,Spa,Bookstore,Boutique,Bubble Tea Shop
2,98033,Kirkland,47.6769,-122.2060,0,Park,Coffee Shop,Sandwich Place,Beach,Thai Restaurant,American Restaurant,Greek Restaurant,Trail,Outdoor Sculpture,Kitchen Supply Store
3,98037,Lynnwood,47.8209,-122.3151,1,Coffee Shop,Mexican Restaurant,Grocery Store,Rental Car Location,Pizza Place,BBQ Joint,Pharmacy,Park,Chinese Restaurant,Sushi Restaurant
4,98040,Mercer Island,47.5707,-122.2221,4,Locksmith,Pharmacy,Bank,Boat Launch,Coffee Shop,Yoga Studio,Home Service,Sandwich Place,Salon / Barbershop,Pizza Place


### Excellent, we now have the clusters and their most common venues for each neighborhood! Let's plot this data on a Folium map:

In [35]:
# Create the map object
latitude = 47.6062
longitude = -122.3321

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Assigning colors to the map features:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding markers to the map:
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_merged['latitude'], seattle_merged['longitude'], seattle_merged['neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Yay! We now have a map of the Seattle neighborhoods with cluster labels.

# Let's examine the clusters more closely:

## Cluster 1 Venues:

In [36]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 0, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Federal Way,Mexican Restaurant,Korean Restaurant,Coffee Shop,Grocery Store,Burger Joint,Pet Store,Ice Cream Shop,Department Store,Thai Restaurant,Vietnamese Restaurant
1,Bellevue,Coffee Shop,Yoga Studio,Toy / Game Store,Steakhouse,Shopping Mall,Clothing Store,Spa,Bookstore,Boutique,Bubble Tea Shop
2,Kirkland,Park,Coffee Shop,Sandwich Place,Beach,Thai Restaurant,American Restaurant,Greek Restaurant,Trail,Outdoor Sculpture,Kitchen Supply Store
11,"University District, Laurelhurst",Pizza Place,Park,Grocery Store,Bubble Tea Shop,Bookstore,Coffee Shop,Burger Joint,Ice Cream Shop,Chinese Restaurant,Dessert Shop
19,Northgate,Coffee Shop,Sandwich Place,Video Store,Arts & Crafts Store,Toy / Game Store,Breakfast Spot,Chinese Restaurant,Park,Convenience Store,Movie Theater


### Cluster 1 Observation: 
Cluster 1 features a healthy mix of eateries and coffee shops implying favorable foot traffic and quality of patron. The extraordinary concentration of eateries is encouraging, particularly in the University District which features Coffee Shops as the 6th most common venue. It is a similar story in Federal Way, but Coffee Shops are still the 3rd most common venue, indicating relatively stiff competition.

## Cluster 2 Venues:

In [37]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 1, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Lynnwood,Coffee Shop,Mexican Restaurant,Grocery Store,Rental Car Location,Pizza Place,BBQ Joint,Pharmacy,Park,Chinese Restaurant,Sushi Restaurant
5,Redmond,Coffee Shop,Bakery,Mexican Restaurant,Pizza Place,Park,Sandwich Place,Brewery,Indian Restaurant,Bookstore,Italian Restaurant
6,Renton,Coffee Shop,Sandwich Place,Mexican Restaurant,Sports Bar,Pizza Place,Pharmacy,Sushi Restaurant,Bar,Thai Restaurant,Pet Store
9,"Greenwood, Fremont, Greenlake",Coffee Shop,Pizza Place,Mexican Restaurant,Chinese Restaurant,Bar,Burger Joint,Café,Ice Cream Shop,Bakery,Cocktail Bar
12,Ballard,Coffee Shop,Cocktail Bar,Mexican Restaurant,Bar,Ice Cream Shop,Bakery,French Restaurant,Thai Restaurant,Park,Brewery
15,Madrona,Coffee Shop,Grocery Store,Park,Bakery,Italian Restaurant,French Restaurant,Thai Restaurant,Gym,Pizza Place,Fried Chicken Joint
16,"West Seattle, Alki Beach",Coffee Shop,Pizza Place,Grocery Store,Park,Italian Restaurant,Pub,Spa,Pet Store,Mexican Restaurant,Scenic Lookout


### Cluster 2 Observation: 
Cluster 2 is overwhelmingly populated by coffee shop competitors. The extraordinary concentration of eateries is encouraging, but the competition is equally discouraging. Further analysis is required to ascertain distance to competitors in these neighborhoods; there may yet be a unique opportunity which the coarse geolocation data does not readily support.

## Cluster 3 Venues:

In [38]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 2, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Bainbridge Island,Boat or Ferry,Pizza Place,Coffee Shop,Trail,Bank,Sandwich Place,Ice Cream Shop,Park,Diner,Scenic Lookout
17,Columbia City,Vietnamese Restaurant,Pizza Place,Coffee Shop,Park,Bar,Pub,Gym,Ice Cream Shop,Farmers Market,Italian Restaurant
20,Mount Baker,Park,Pizza Place,Coffee Shop,Gym,Trail,Scenic Lookout,Pub,Playground,Taco Place,Mexican Restaurant
21,Magnolia,Park,Coffee Shop,Harbor / Marina,Playground,Pizza Place,Trail,Japanese Restaurant,Sporting Goods Shop,Seafood Restaurant,Scenic Lookout


### Cluster 3 Observation: 
Cluster 3 is populated by outdoor venues, eateries, and coffee. High concentration of eateries and outdoor activities implies the quality of patronage is favorable, but competition density is significant. Further analysis is required to ascertain distance to competitors; there may yet be a unique opportunity which the coarse geolocation data does not readily support.

## Cluster 4 Venues:

In [39]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 3, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown,Hotel,Coffee Shop,Bakery,Sandwich Place,Breakfast Spot,Brewery,New American Restaurant,Fish Market,Middle Eastern Restaurant,Seafood Restaurant
8,Capitol Hill,Coffee Shop,Bakery,Cocktail Bar,Café,Italian Restaurant,Sandwich Place,Park,Yoga Studio,Museum,Bar
10,"International District, Pioneer Square",Coffee Shop,Hotel,Bakery,Brewery,Vietnamese Restaurant,Breakfast Spot,Pizza Place,Sushi Restaurant,Seafood Restaurant,Sandwich Place
13,"South Lake Union, Queen Anne",Coffee Shop,Hotel,Yoga Studio,Bakery,Sandwich Place,Taco Place,Museum,Gym,Gym / Fitness Center,Middle Eastern Restaurant
18,Belltown,Hotel,Coffee Shop,Bakery,Movie Theater,Breakfast Spot,Cocktail Bar,Sculpture Garden,Deli / Bodega,Middle Eastern Restaurant,Scenic Lookout


### Cluster 4 Observation: 
Cluster 4 is heavility populated by coffee shops and hotels, in addition to bakeries, which, too, regularly offer coffee as an option. High concentration of eateries implies the quality of patronage is favorable; competition is not favorable in any of the cluster 4 neighborhoods.

## Cluster 5 Venues:

In [40]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 4, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Mercer Island,Locksmith,Pharmacy,Bank,Boat Launch,Coffee Shop,Yoga Studio,Home Service,Sandwich Place,Salon / Barbershop,Pizza Place


### Cluster 5 Observation:
Cluster 5 features a lone neighborhood, Mercer Island. Non-culinary establishments imply poor foot traffic. Furthermore, Mercer Island is not a particularly accessible neighborhood, requiring a toll fee to access, coupled with a single access point on both the East and West side of the Puget Sound. This increased cost of both time and money produces an additional point of friction for patronage. 

### Final Recommendation: 
Seattle’s University District is a clear favorite for establishing a new coffee shop. This observation is based on relatively minimal competition coupled with high foot traffic to other non-competitor eateries. Federal Way is a not-too-distant runner up with similar qualities, which a slightly higher density of competition.

Further analysis utilizing finer-grain geolocation data and foot traffic data is warranted for most other neighborhoods, save for Mercer Island, which presents multiple points of friction.